Import modules

In [1]:
import geopandas as gpd
import numpy as np
import imageio
import os
import itertools
import shutil
import argparse

from matplotlib import pyplot, cm, colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from multiprocess import Pool, cpu_count

import utils

In [2]:
parser = argparse.ArgumentParser(description="Creates video animations from COVID daily reports")
parser.add_argument("--minimal", "--min", action='store_true', default=False, help="Create only necessary animations")

args = parser.parse_known_args()


Prepare environment and load data

In [3]:
world_shape_df=gpd.read_file('maps/ne_10m_admin_0_sovereignty.shp')
world_states_shape_df = gpd.read_file('maps/ne_10m_admin_1_states_provinces.shp')

russia_shape_df = None if args[0].minimal else world_states_shape_df[world_states_shape_df.admin == 'Russia']

world_report_df = utils.storage.get_countries_report()
russia_report_df = None if args[0].minimal else utils.storage.get_regions_report('Russia')

world_report_df.reset_index(inplace=True)
if russia_report_df is not None:
    russia_report_df.reset_index(inplace=True)

if not os.path.exists('./assets'):
    os.mkdir('./assets')
    os.mkdir('./assets/video')
elif not os.path.exists('./assets/video'):
    os.mkdir('./assets/video')
    
if not os.path.exists('./temp'):
    os.mkdir('./temp')

Process data - add more metrics if needed

In [4]:
world_stats = utils.storage.get_countries_stats()

def augment_report(df, country, region = None):
    columns = ['Confirmed', 'Deaths', 'Confirmed_Change', 'Deaths_Change', 'Rt']

    name = country
    if region:
        name = region
        
    if name == 'Main territory':
        df.drop(df[df.Name == name].index ,inplace=True)
        return
        
    for column in columns:
        values =  df.loc[df.Name == name, ['Date', column]].sort_values(by='Date')[column]
        
        for sma in [3, 5, 7, 10, 14]:
            df.loc[df.Name == name, column + '_SMA_' + str(sma)] = values.rolling(window=sma).mean()
            
        if column == 'Rt':
            continue
        
        df.loc[df.Name == name, column + '_Norm'] = utils.data.normalize(values)

        values_per_capita = utils.data.per_capita(values, country, region)
        for per in [1, 1_000, 100_000]:
            suffix = 'per_capita'
            if per == 1000:
                suffix = 'per_1k'
            elif per == 100_000:
                suffix = 'per_100k'
            
            df.loc[df.Name == name, column + '_' + suffix] = values_per_capita * per
    
    

for country in tqdm(utils.storage.get_countries(), desc='Add new columns to world report'):
    augment_report(world_report_df, country)
    world_report_df.loc[world_report_df.Name == country, 'Continent'] = world_stats.loc[country, 'Continent']

if not args[0].minimal:
    for region in tqdm(utils.storage.get_country_regions('Russia'), desc='Add new columns to russia report'):
        augment_report(russia_report_df, 'Russia', region)

Add new columns to russia report: 100%|██████████| 86/86 [00:08<00:00, 10.70it/s]


In [5]:
russia_report_df

,index,Date,Confirmed,Active,Recovered,Deaths,Confirmed_Change,Active_Change,Recovered_Change,Deaths_Change,...,Deaths_Change_SMA_14,Deaths_Change_Norm,Deaths_Change_per_capita,Deaths_Change_per_1k,Deaths_Change_per_100k,Rt_SMA_3,Rt_SMA_5,Rt_SMA_7,Rt_SMA_10,Rt_SMA_14
324,0,2020-01-31,0,0,0,0,0,0,0,0,...,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
325,1,2020-02-01,0,0,0,0,0,0,0,0,...,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
326,2,2020-02-02,0,0,0,0,0,0,0,0,...,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
327,3,2020-02-03,0,0,0,0,0,0,0,0,...,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
328,4,2020-02-04,0,0,0,0,0,0,0,0,...,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27859,319,2020-12-15,19252,1051,18079,122,188,70,114,4,...,2.642857,0.666667,0.000003,0.003160,0.316035,1.005755,1.006389,1.006282,1.007772,1.006039
27860,320,2020-12-16,19444,1101,18218,125,192,50,139,3,...,2.642857,0.500000,0.000002,0.002370,0.237026,1.004424,1.006120,1.005323,1.006553,1.006613
27861,321,2020-12-17,19637,1118,18389,130,193,17,171,5,...,2.785714,0.833333,0.000004,0.003950,0.395043,1.005295,1.005302,1.005884,1.005859,1.007468
27862,322,2020-12-18,19827,1126,18565,136,190,8,176,6,...,3.000000,1.000000,0.000005,0.004741,0.474052,1.005724,1.005825,1.006636,1.006115,1.007454


Rename some countries and regions or remove them to join two shape and data dataframes

In [6]:
for data, shape in [
    ('North Macedonia', 'Macedonia'),
    ('Holy See', 'Vatican'),
    ('Cote d\'Ivoire', 'Ivory Coast'),
    ('Congo (Kinshasa)', 'Democratic Republic of the Congo'),
    ('Congo (Brazzaville)', 'Republic of the Congo'),
    ('Bahamas', 'The Bahamas'),
    ('Serbia', 'Republic of Serbia'),
    ('Sao Tome and Principe', 'São Tomé and Principe'),
    ('Tanzania', 'United Republic of Tanzania'),
    ('UK', 'United Kingdom'),
    ('US', 'United States of America')
]:
    world_report_df.loc[world_report_df.Name == data, 'Name'] = shape
    
for to_remove in ['West Bank and Gaza', 'Timor-Leste']:
    world_report_df = world_report_df[world_report_df.Name != to_remove]
    
world_shape_df.loc[world_shape_df['ADMIN'] == 'Baykonur Cosmodrome', 'ADMIN'] = 'Kazakhstan'


if not args[0].minimal:
    russia_shape_df.loc[1442, 'name_ru'] = "Алтайский край"
    russia_shape_df.dropna(subset = ['name_ru'], inplace = True)
    
    for data, shape in [
        ('Крым', 'Автономная Республика Крым'),
        ('Алтай', 'Республика Алтай'),
        ('Еврейская АО', 'Еврейская автономная область'),
        ('Карачаево-Черкессия', 'Карачаево-Черкесия'),
        ('Карелия', 'Республика Карелия'),
        ('Коми', 'Республика Коми'),
        ('Ненецкий АО', 'Ненецкий автономный округ'),
        ('Северная Осетия', 'Республика Северная Осетия-Алания'),
        ('Саха (Якутия)', 'Якутия'),
        ('ХМАО – Югра', 'Ханты-Мансийский автономный округ — Югра'),
        ('Чукотский АО', 'Чукотский автономный округ'),
        ('Ямало-Ненецкий АО', 'Ямало-Ненецкий автономный округ'),
    ]:
        russia_shape_df.loc[russia_shape_df.name_ru == shape, 'name_ru'] = data
        
    russia_shape_df['name_ru'] = russia_shape_df['name_ru'].apply(lambda x: x[:-4] +'.' if x.endswith('область') else x)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
<ipython-input-6-300d2728652f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  russia_shape_df.dropna(subset = ['name_ru'], inplace = True)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Prepare shapes dataframes

In [7]:
world_shape_df = world_shape_df.to_crs('epsg:4326')
europe_shape_df = world_shape_df.loc[world_shape_df.ADMIN.isin(set(world_report_df.loc[world_report_df.Continent=='Europe','Name']))]

if not args[0].minimal:
    russia_shape_df = russia_shape_df.to_crs('epsg:5940')   

Preparations for images rendering:
- Set fin, step variables
- Set pool_size. CAUTION: Big values could lead to out of memory exceptions and to kernel crash
- Set cmap

In [8]:
step = utils.one_day
fin =  utils.last_day
pool_size = max(1, int(cpu_count()*.75))
sea_color = '#CBE8FE'

cmap_general = colors.LinearSegmentedColormap.from_list('test',
 [
     (0,'#00cc00'),
     (0.2,'#28a428'),
     (0.4,'#7ba428'), 
     (0.5,'#ccad00'),
     (0.7,'#e69500'),
     (0.9,'#cc3600'), 
     (1,'#ba1234')
 ])

cmap_deaths = colors.LinearSegmentedColormap.from_list('test',
 [
     (0,'#e9edee'),
     (0.1,'#ffff91'),
     (0.2,'#ff713f'), 
     (0.5,'#a42c2b'),
     (0.7,'#595959'),
     (1,'#000000')
 ])

Define helper functions

In [9]:
fig,ax = None, None

def get_fig_axes(cmap, norm):
    fig = pyplot.figure(figsize=(24,14))
    fig.set_tight_layout({"pad":0.1})
        
    ax = fig.add_subplot(1,1,1)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="1%", pad=0.05)
    fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax,cax=cax,orientation='horizontal')
   
    return fig, ax

def draw_top_10(df, name_column, data_column, ax, 
                name_name='Country', data_name = 'Cases',
                as_int = True, x0 = 0.01, y0 = 0.01):
    formatter = (lambda x: '{:d}'.format(int(x))) if as_int else (lambda x: '{:.2f}'.format(float(x)))
    table_data =list(
        df.dropna(subset=[data_column]).sort_values(
            by = data_column,
            ascending = False).head(10)[[name_column, data_column]].apply(
            lambda x: [x[name_column], formatter(x[data_column])], axis=1).values)
    
    count = len(table_data)
    
    if count > 0:
        table = ax.table(table_data,
                         colLabels=[name_name, data_name],
                         rowLabels=list(range(1, count+1)),
                         colWidths=[2, 1],
                         cellColours=np.reshape(np.repeat(sea_color, 2*count), (count, 2)),
                         rowColours = np.repeat(sea_color, count),
                         colColours = np.repeat(sea_color, 2),
                         bbox=[x0, y0, .2, .027*(count+1)])
        
        table.auto_set_font_size(False)
        table.set_fontsize(10)    

def process_day (day,
                 shape_df,
                 report_df,
                 column_name,
                 vmin,
                 vmax,
                 cmap,
                 norm,
                 annotation_text,
                 folder_name,
                 frame_title = None,
                 ax_xlim = (-180, 180),
                 ax_ylim = (-90,90),
                 annotation_table_column_name = 'Country',
                 annotation_table_column_data = 'Cases',
                 annotation_table_data_as_int = True,
                 annotation_table_x0 = 0.01,
                 annotation_table_y0 = 0.01,
                 shape_df_index = 'ADMIN', 
                 report_df_index = 'Name'):
    global fig, ax
    if not fig:
        fig,ax = get_fig_axes(cmap, norm)
        
    if ax_xlim:
        ax.set_xlim(ax_xlim[0],ax_xlim[1])
        
    if ax_ylim:
        ax.set_ylim(ax_ylim[0],ax_ylim[1])
        
    ax.set_axis_off()
    
    if frame_title:
        fig.suptitle(frame_title, fontsize = 36)
        
    temp_df = shape_df.set_index(shape_df_index).join(report_df[report_df.Date == day].set_index(report_df_index)[column_name]).reset_index()
        
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation_text, xy=(20,950), xycoords='figure pixels', fontsize=24)
    draw_top_10(temp_df, 
                shape_df_index,
                column_name, 
                ax, 
                name_name = annotation_table_column_name,
                data_name = annotation_table_column_data,
                as_int = annotation_table_data_as_int,
                x0 = annotation_table_x0,
                y0 = annotation_table_y0)
    
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor=sea_color)    
    ax.clear()


def make_video(name, clean_data = True):
    images = list(os.listdir(f'./temp/{name}'))

    with imageio.get_writer(f'./assets/video/{name}.mp4', mode='I', fps=6) as writer:
        for i in range(len(images)):
            image = imageio.imread(os.path.join(os.path.abspath(f'./temp/{name}'),images[i]))
            writer.append_data(image)
            
    if clean_data:
        shutil.rmtree(f'./temp/{name}')
        

        

data_selector = {
    'world' : (
        world_shape_df,
        world_report_df,
        utils.first_day,
        (-180, 180),
        (-90, 90),
        (0.01, 0.1)
    ),
        
    'europe' : (
        europe_shape_df,
        world_report_df,
        utils.str_to_datetime('01-02-2020'),
        (-20, 50),
        (30, 73),
        (-0.05, 0.01)
    ),
        
    'russia' : (
        russia_shape_df, 
        russia_report_df,
        utils.str_to_datetime('15-03-2020'),
        None,
        None,
        (0.01, 0.01)
    )
}
        
get_shape_index = lambda shape: "name_ru" if shape == 'russia' else 'ADMIN'
get_as_int = lambda column, suffix: column != 'Rt' and suffix == ''
get_table_column_name = lambda shape: "Region" if shape == 'russia' else "Country"
get_folder_name = lambda shape, column, suffix: (f'{shape}_{column}_{suffix}' if suffix else f'{shape}_{column}').lower()
get_cmap = lambda column: cmap_deaths if (column == 'Deaths') or (column == 'Deaths_Change') else cmap_general

def get_table_column_data (column, suffix):
    if suffix == '100k':
        return 'Cases per 100k'
    elif suffix == 'Norm':
        return 'Fraction from max'
    elif suffix.startswith("SMA"):
        return 'Sliding mean avg'
    elif column == 'Rt':
        return r'$R_t$'
    
    return 'Cases'

def get_column_name (column, suffix):
    if suffix == '100k':
        return column + '_per_100k'
    elif suffix == 'Norm' or suffix.startswith("SMA"):
        return f'{column}_{suffix}'
    
    return column

def get_frame_title(shape, column, suffix):
    if shape == 'russia':
        shape_part = 'Regions'
    else:
        shape_part = 'Countries'
    
    if column == 'Rt':
        column_part = r'$R_t$' 
    elif column == 'Confirmed_Change':
        column_part = 'new cases per day'
    elif column == 'Deaths_Change':
        column_part = 'deaths per day'
    else:
        column_part = column.lower()
        
    if suffix == '100k':
        suffix_part = 'per 100k people'
    elif suffix == 'Norm':
        suffix_part = 'normalized by min-max'
    elif suffix.startswith('SMA'):
        suffix_part = f'sliding mean average for {suffix[4:]} days'
        
    return f'{shape_part} {column_part} {suffix_part}' if suffix != '' else f'{shape_part} {column_part}'

def get_vmin_vmax(report_df, column, suffix):
    column_name = get_column_name(column, suffix)
    if (suffix == 'Norm'):
        return 0, 1
    
    elif (column == 'Rt'):
        return 0, 2
    
    elif (column == 'Confirmed_Change' or column == 'Deaths_Change'):
        max_values = list()
        
        for name in set(report_df.Name):
            name_values = report_df.loc[report_df.Name == name, column_name]
            max_value = name_values.dropna().max()
            if not np.isnan(max_value):
                max_values.append(max_value)
            
        return max(1, np.quantile(max_values, .05)), np.quantile(max_values, .95)

    return max(1, report_df.loc[report_df.Date == fin, column_name].quantile(.05)), report_df.loc[report_df.Date == fin, column_name].quantile(.95)

def get_norm(column, suffix, vmin, vmax):
    if (suffix == 'Norm'):
        return colors.Normalize(vmin, vmax)
    elif (column == 'Rt'):
        return colors.Normalize(.9, 1.1)
    
    return colors.LogNorm(vmin, vmax)
            

    
    

def generate_frames(shape, column, suffix, idx, total):    
    shape_df, report_df, start, ax_xlim, ax_ylim, table_offset = data_selector[shape]
    column_name = get_column_name (column, suffix)
    vmin, vmax = get_vmin_vmax (report_df, column, suffix)
    cmap = get_cmap (column)
    norm = get_norm (column, suffix, vmin, vmax)
    folder_name = get_folder_name (shape, column, suffix)
    frame_title = get_frame_title (shape, column, suffix)
    table_column_name = get_table_column_name (shape)
    table_column_data = get_table_column_data (column, suffix)
    as_int = get_as_int (column, suffix)
    shape_index = get_shape_index (shape)
    
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')

##    if True:
    with Pool(pool_size) as frames_pool:
        frames = list()

        for i in range ((fin - start).days + 1):
            day = start + step*i
            annotation_text = day.date().strftime('%Y-%m-%d')

            frames.append(frames_pool.apply_async(
                process_day,
                [
##            process_day(
                    day,
                    shape_df,
                    report_df,
                    column_name,
                    vmin,
                    vmax,
                    cmap,
                    norm,
                    annotation_text,
                    folder_name,
                    frame_title,
                    ax_xlim,
                    ax_ylim,
                    table_column_name,
                    table_column_data,
                    as_int,
                    table_offset[0],
                    table_offset[1],
                    shape_index
                ]))
##)

        for frame in tqdm(frames, desc=f'({idx}/{total}) Frames for {shape} - {column_name}'): 
            frame.wait()
        
    return folder_name

In [10]:
#FOR DEBUG ONLY
#generate_frames('europe','Deaths_Change','SMA_7',1,1)

Make videos

In [11]:
if __name__ == '__main__':
    shapes = ['world', 'europe'] if args[0].minimal else ['world', 'europe', 'russia']
    columns = ['Confirmed', 'Deaths'] if args[0].minimal else ['Confirmed', 'Deaths', 'Confirmed_Change', 'Deaths_Change']
    suffixes = ['', '100k'] if args[0].minimal else ['', '100k', 'Norm']
    
    all_variants = list(itertools.product(shapes, columns, suffixes))

    with Pool(1) as videos_pool:
        videos = list()
        idx = 1
    
        for shape, column, suffix in all_variants:
            folder = generate_frames(shape, column, suffix, idx, len(all_variants))
            videos.append(videos_pool.apply_async(make_video, [folder]))
            idx = idx + 1
        
        for video in tqdm(videos, desc="Videos rendered"): 
            video.wait()

(7/36) Frames for world - Confirmed_Change: 100%|██████████| 333/333 [01:38<00:00,  3.38it/s]
(8/36) Frames for world - Confirmed_Change_per_100k: 100%|██████████| 333/333 [01:41<00:00,  3.28it/s]
(10/36) Frames for world - Deaths_Change: 100%|██████████| 333/333 [01:43<00:00,  3.21it/s]
(11/36) Frames for world - Deaths_Change_per_100k: 100%|██████████| 333/333 [01:39<00:00,  3.35it/s]
(19/36) Frames for europe - Confirmed_Change: 100%|██████████| 323/323 [01:03<00:00,  5.08it/s]
(20/36) Frames for europe - Confirmed_Change_per_100k: 100%|██████████| 323/323 [01:05<00:00,  4.91it/s]
(21/36) Frames for europe - Confirmed_Change_Norm: 100%|██████████| 323/323 [01:00<00:00,  5.37it/s]
(22/36) Frames for europe - Deaths_Change: 100%|██████████| 323/323 [01:01<00:00,  5.27it/s]
(23/36) Frames for europe - Deaths_Change_per_100k: 100%|██████████| 323/323 [01:00<00:00,  5.31it/s]
(31/36) Frames for russia - Confirmed_Change: 100%|██████████| 280/280 [00:26<00:00, 10.55it/s]
(32/36) Frames fo